In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import os
import pandas as pd
import numpy as np

import financial.data as fd
from financial.io.file.cache import FileCache
from financial.io.cache import NoUpdateStrategy
from financial.momentum.tuning.oosTools import quick_oos_metrics

In [43]:
clenow = pd.read_csv("/home/manidmt/TFG/OTRI/models/scikit-learn/scikit-learn_clenow_^GSPC_2025_preds.csv", index_col=0, parse_dates=True)
clenow
start_date = "2015-01-01"

In [44]:
clenow_short = clenow.loc[start_date:]
clenow_short

,0
2015-01-02,2109.108981
2015-01-05,2098.735067
2015-01-06,2100.853203
2015-01-07,2065.583583
2015-01-08,2062.263811
...,...
2025-06-24,6288.359683
2025-06-25,6536.380303
2025-06-26,6444.701360
2025-06-27,6477.057143


In [55]:
clenow_short_abs = pd.read_pickle("/home/manidmt/TFG/OTRI/cache/model-momentum-clenow_^GSPC_2025@pred").loc[start_date:]
clenow_short_abs

2015-01-02    0.010959
2015-01-05    0.031255
2015-01-06    0.040885
2015-01-07    0.028745
2015-01-08    0.011112
                ...   
2025-06-24   -0.044452
2025-06-25   -0.038746
2025-06-26   -0.040701
2025-06-27   -0.039462
2025-06-30   -0.037862
Length: 2639, dtype: float64

In [45]:
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=os.environ["CACHE"] + "/", update_strategy=NoUpdateStrategy()))
data = datastore.get_data("^GSPC", start, "2025-06-30")
data

2015-01-02    2058.199951
2015-01-05    2020.579956
2015-01-06    2002.609985
2015-01-07    2025.900024
2015-01-08    2062.139893
                 ...     
2025-06-24    6092.180000
2025-06-25    6092.160000
2025-06-26    6141.020000
2025-06-27    6173.070000
2025-06-30    6204.950000
Name: ^GSPC, Length: 2639, dtype: float64

In [56]:
clenow_short.index = pd.to_datetime(clenow_short.index)
data.index = pd.to_datetime(data.index)

df = pd.DataFrame({
    "pred": clenow_short_abs,
    "true": data
})

In [57]:
df["y_true"] = (df["true"].shift(-20) - df["true"]) / df["true"]
df = df.dropna()     

In [58]:
err  = df["pred"] - df["y_true"]
mae  = err.abs().mean()
rmse = np.sqrt((err**2).mean())
r2   = 1 - ((err**2).sum() / ((df["y_true"] - df["y_true"].mean())**2).sum() + 1e-12)
corr = df["pred"].corr(df["y_true"])
hit  = (np.sign(df["pred"]) == np.sign(df["y_true"])).mean()

In [60]:
print(f"  Mean Absolute Error (MAE):       {mae:.5f}")
print(f"  Root Mean Square Error (RMSE):   {rmse:.5f}")
print(f"  R² Score:                        {r2:.5f}")
print(f"  Correlation (pred vs true):      {corr:.5f}")
print(f"  Hit Ratio (same direction):      {hit:.5%}")

  Mean Absolute Error (MAE):       0.04481
  Root Mean Square Error (RMSE):   0.06318
  R² Score:                        -1.02564
  Correlation (pred vs true):      0.01642
  Hit Ratio (same direction):      53.76098%
